In [ ]:
!pip install pennylane

In [ ]:
import pennylane as qml
import numpy as np

n_qubits = 6
n_layers = 3

dev = qml.device("default.qubit", wires=n_qubits)

def feature_map(x):
    for i in range(n_qubits):
        qml.Hadamard(wires=i)
        qml.RY(x[i], wires=i)
        qml.RZ(x[i], wires=i)

def variational_ansatz(params):
    for l in range(n_layers):
        for i in range(n_qubits):
            qml.RY(params[l * n_qubits + i], wires=i)
            qml.CNOT(wires=[i, (i+1) % n_qubits])
            qml.RZ(params[l * n_qubits + i], wires=i)
        qml.Barrier()

@qml.qnode(dev)
def quantum_kernel(x1, x2, params):
    feature_map(x1)
    variational_ansatz(params)
    qml.adjoint(feature_map)(x2)
    return qml.expval(qml.PauliZ(0))
    
def cost(params, X):
    N = len(X)
    kernel_matrix = np.zeros((N, N))

    for i in range(N):
        for j in range(N):
            kernel_matrix[i, j] = float(quantum_kernel(X[i], X[j], params))  # Ensure it's a scalar value
    
    return np.sum(kernel_matrix)


In [ ]:
from scipy.optimize import minimize
from tqdm import tqdm

# Cost function
def cost(params, X):
    N = len(X)
    kernel_matrix = np.zeros((N, N))
    
    for i in tqdm(range(N)):
        for j in range(N):
            kernel_matrix[i, j] = qml.math.toarray(quantum_kernel(X[i], X[j], params))
    
    return np.sum(kernel_matrix)

# Sample Data
X = np.random.rand(20, n_qubits)  # 20 samples with n_qubits features
params = np.random.rand(n_layers * n_qubits)

# Optimization
res = minimize(lambda p: cost(p, X), params.flatten(), method='COBYLA', options={'maxiter': 20})

In [ ]:
from sklearn.cluster import KMeans

# Compute quantum kernel matrix
kernel_matrix = np.zeros((len(X), len(X)))
for i in range(len(X)):
    for j in range(len(X)):
        kernel_matrix[i, j] = qml.math.toarray(quantum_kernel(X[i], X[j], params))

# Apply k-means clustering to quantum feature space
kmeans = KMeans(n_clusters=3)
kmeans_labels = kmeans.fit_predict(kernel_matrix)

In [ ]:
from sklearn.metrics import silhouette_score

silhouette_classical = silhouette_score(X, kmeans_labels)
silhouette_quantum = silhouette_score(kernel_matrix, kmeans_labels)

print("Silhouette Score (Classical):", silhouette_classical)
print("Silhouette Score (Quantum):", silhouette_quantum)